# Basic Foyer tutorial

### _Foyer is a tool for atomtyping molecular systems._

The second of the MoSDeF tools we will explore is the [Foyer package](https://github.com/mosdef-hub/foyer).  

<p align="center">
    <img src="../graphics/foyer-mosdef.svg", alt="GMSO within the MoSDeF Ecosystem" width="500" height="500"/>
</p>

Foyer can be used to take a molecular model featuring particle positions and bonds (such as an mBuild Compound) and automatically parameterize the system, as directed by a user-specified force field.

The Foyer software itself is designed to be force field agnostic:
* allows Foyer to work with  different force fields without the need to change the software itself (e.g., TraPPE, OPLS, IFF) 
* parameters and their usage rules are encoded in an XML file that is read and evaluated by the Foyer code

By encoding parameters and their usage rules in separate XML files, it makes it easier to:
* disseminate force fields
* modify existing force fields
* track history using version control
* tag/cite specific versions used in research/publications 
* the XML file format is both human and machine readable


Because these files are separate from the code itself, it is easy to make custom force field files for specific applications that deviate from the "canonical" force fields, while ensuring that such modifications are clearly captured and can be used by others. It also makes it easier to application/molecule specific force fields that are easier to validate and extended. 


In this tutorial, we'll demonstrate how Foyer works by using it to perform atom-typing and parameterization of several different molecules.

-------

## Classical Force Fields

Classical force fields, which describe the interaction between atoms, typically contain numerous fitting parameters (e.g., Lennard-Jones sigma and epsilon, partial charge), allowing them to be tuned to describe the behavior of atoms in various chemical environments (i.e., `chemical context`).  

For example, consider carbon in various local bonded environments of the following, along with their fitting parameters for OPLS (units, epsilon: kJ/mol, sigma: nm, charge: multiple of electron charge): 

- #### A terminal methyl group in an alkane, i.e., C-CH3
    * opls_135
    * charge=-0.18
    * sigma=0.35 
    * epsilon=0.276144
- #### A terminal fluorinated methyl group, i.e., C-CF3
    * opls_961
    * charge=0.36
    * sigma=0.35  
    * epsilon=0.276144
- #### As a terminal group in an alkene, i.e., C=CH2
    * opls_143
    * charge=-0.23
    * sigma=0.355
    * epsilon=0.317984

Even though the element does not change (i.e., we are dealing with carbon in each case), the force field fitting parameters are different for OPLS (and would be for many other common force fields).  As such, force field databases may contain 10s or 100s of parameters for a given element, where each unique set of parameters is referred to as an 'atom-type'; i.e., defining the element in a different `chemical context`.

Properly determining the correct set of parameters for a given element (i.e., identifying which atom-type should be used), can be a difficult and error-prone task, but essential for the final results of a simulation to be correct.   

Foyer has been developed as both a scheme to encode these rules for atom-typing along with a software that evaluates these rules. Here we will first focus on the scheme to encode rules and the associated file format. 

-------

## Foyer XML format

Foyer forcefields are defined within an XML file that contains both the 'rules' required for atomtyping as well as the forcefield parameters within a single file. 

The Foyer XML format is an extension of the [OpenMM forcefield XML format](http://docs.openmm.org/latest/userguide/application/05_creating_ffs.html).  The only differences reside in the `AtomTypes` section, where several additional attributes are available for each `Type`.  

Let's take a look at the basic structure of a Foyer XML file.

### AtomTypes

In [ ]:
! sed -n 2,18p xml_files/OPLSaa_alkanes.xml

The `AtomTypes` section of the Foyer XML is similar to that used for OpenMM forcefield XMLs; however, each `Type` in Foyer XML supports four additional attributes not found in OpenMM:
* `def` - SMARTS string describing the chemical substructure of this atomtype (Follow [this link](https://github.com/mosdef-hub/foyer/issues/63) for more info on SMARTS-based atomtyping in Foyer, i.e., what parts of the language are supported)
* `desc` - Brief textual description of the atom type
* `doi` - DOI reference for parameters associated with this atom type (if available)
* `overrides` - One or more atom types to 'override'l; this defines rule precedence

Atom types are defined using [SMARTS](http://www.daylight.com/dayhtml/doc/theory/theory.smarts.html), which provide a language for describing chemical structures and substructures.

### Forcefield parameters

The remaining sections of the Foyer XML are currently the same as defined in the OpenMM format and feature both bonded and nonbonded forcefield parameters.

In [ ]:
! sed -n 19,42p xml_files/oplsaa_alkanes.xml

-------

## Defining [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html)

Let us consider what each of the SMARTS strings defined in the file tells us:

`opls_135`, SMARTS string, `def="[C;X4](H)(H)H"`
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- At least 3 of those neighbors are hydrogens, i.e., `(H)(H)H`

`opls_138`,  SMARTS string, `def="[C;X4](H)(H)(H)H"`
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- 4 of those neighbors are hydrogens, i.e., `(H)(H)(H)H`

`opls_136`,  SMARTS string, `def="[C;X4](H)H"`
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- At least 2 of those neighbors are hydrogens, i.e., `(H)H`

`opls_140`,  SMARTS string, `def="H[C;X4]"`
- The element is hydrogen, i.e., `H`
- When that hydrogen is connected to a carbon that has 4 neighbors, i.e., `[C;X4]`
    * note, SMARTS allows us to encode chemical context information about our neighbors as well (i.e., encoding more than just our first neighbor shell)


## Atomtyping using SMARTS
Let us now consider using these rules to atom-type the carbon in methane (i.e., `[C;X4](H)(H)(H)H`), using the following rules (reproduced from the xml/oplsaa_alkanes.xml file):

```
    <AtomTypes>
        <Type name="opls_135" def="[C;X4](H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH3"
              doi="10.1021/ja9621760" overrides="opls_136"/>

        <Type name="opls_136" def="[C;X4](H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH2"
              doi="10.1021/ja9621760"/>

        <Type name="opls_138" def="[C;X4](H)(H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH4"
              doi="10.1021/ja9621760" overrides="opls_135,opls_136"/>

        <Type name="opls_140" def="H[C;X4]"
              class="HC" element="H" mass="1.00800" desc="alkane H"
              doi="10.1021/ja9621760"/>
    </AtomTypes>
    
```

- `opls_138` would obviously evaluate to `True`, as it is defined for a carbon, with 4 hydrogen neighbors. 
- `opls_135` and `opls_136` would also evaulate to `True`.  
    * In the case of `opls_135`, our definition only states that at least 3 of the neighbors are hydrogen, and have not made any specific claims about the identity of the 4th neighbor.
    * Similarly, for opls_136, we have only stated that 2 neighbors must be hydrogen and not specified the identity of the other 2 neighbors. 

## If 3 out of 4 rules evaluated to `True`...which one do we use?  
* We will discuss two ways to address this in Foyer. 



#### Notes: 
* Foyer will evaluate all the rules in a forcefield file, rather than just stopping at the first rule that evaluates to `True`
    * This allows rules to be defined in any order
* Foyer iterates over the rules (iterating until no changes our found)
    * This allows recursive definitions of usage, i.e., referring to specific atom-types in the SMARTS string, as will be discussed later 





### `overrides`  
`overrides` provide a means to dictate rule precedence (i.e., which rules are more specific than others).  

In the force field file above, `opls_138` has defined: `overrides="opls_135,opls_136"`:  
 * That is, if `opls_138` evaluates to `True`, then it takes precedence over `opls_135` and `opls_136`, even if they evaluate to `True`. 

Similarly, if opls_135 evaluates to `True`, it `overrides="opls_136"`, because opls_135 is more specific, thus taking precedence. 

### More specific SMARTS definitions


#### Recursive definitions 
`overrides` are especially useful if the chemical context of two different atom-types are effectively the same.  E.g., the terminal methyl group in an alkane has the same first neighbor environment as the methyl group in toluene, however the parameters (and thus atom-type) are different. 

Thus `overrides` can be used to state that if the more specific toluene rule evaluates to `True` it should take precedences over the more general alkane rule. Also note that in the SMARTS for toluene, we use a recursive definition.  That is, we use an atom-type name in addition to the element, i.e., `[C;X4](C;%opls_145)(H)(H)H`.  This states that the carbon we are connected to must be opls_145 to evaluate to true.  We can use these recursive definitions because atom-typing is performed as an iterative process in Foyer. Obviously, if we use recursive definitions for all of our SMARTS, we will not be able to atom-type.  

<img src="../graphics/ch3-toluene.png" alt="toluene" style="width: 700px;"/>


#### Longer range patterns
In many cases, recursive definitions can be avoided by providing more specific SMARTS strings.  For example, we could instead take note of the fact that opls_145 only has 3 connections:

`[C;X4](C;X3)(H)(H)H`

We could also state the identify of those connections if necessary (or even define then entire opls_145 SMARTS pattern, `[C;X3;r6]1[C;X3;r6][C;X3;r6][C;X3;r6][C;X3;r6][C;X3;r6]1`), although the larger definitions may begine to be less clear. 

Often times more specific definitions can remove the need to define `overrides`.  For example, the rules for `opls_135`, `opls_136`, and `opls_138` above can be made more specific by stating the identify of the other neighbors and thus eliminate the need for `overrides`, as shown below.  It is important to note that some force fields are designed such to have "generic" parameters that we would default to if more specific cases are not available. 

```
<AtomTypes>
        <Type name="opls_135" def="[C;X4](H)(H)(H)C"
              class="CT" element="C" mass="12.01100" desc="alkane CH3"
              doi="10.1021/ja9621760"/>

        <Type name="opls_136" def="[C;X4](H)(H)(C)C"
              class="CT" element="C" mass="12.01100" desc="alkane CH2"
              doi="10.1021/ja9621760"/>

        <Type name="opls_138" def="[C;X4](H)(H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH4"
              doi="10.1021/ja9621760"/>

        <Type name="opls_140" def="H[C;X4]"
              class="HC" element="H" mass="1.00800" desc="alkane H"
              doi="10.1021/ja9621760"/>
    </AtomTypes>

```

### Applying a force field to atomtype a system
With a force field file defined, we will apply it to an alkane system.  Here, we will use the routines from mBuild  to construct a propane molecule, then perform atom-typing as part of saving our configuration to a lammps data file. 

In [ ]:
import foyer 
from foyer import Forcefield

import mbuild as mb

compound = mb.load("CCC", smiles=True)
compound.visualize()

Let us save to gromacs formatted top file and then examine the output.  To do this, we will call the save operator on the compound, passing the force field information.  

In [ ]:
compound.save('test.top', forcefield_files="xml_files/oplsaa_alkanes.xml", overwrite=True)

!cat test.top 

We can examine the save function using help to find out more.

In [ ]:
help(mb.Compound.save)

Note, we can also use a different syntax to perform atom-typing that does not explicitly rely upon mbuild.

Below, we can load the force field file, and use the apply command, passing to it, in this case, the mbuild compound although this will also support a parmed structure object. 

In the current, non-GMSO version of Foyer, the apply command will return a ParmEd topology; as such, writing to a file will be limited to the writers that are part of ParmEd (e.g., gromacs, openmm, etc.) and not provide access to those that are part of mbuild (e.g, lammps, hoomd, etc.).  The save command in mbuild provides access to both sets. Since this example writes a GROMACS file, we should have no issues.

In [ ]:
oplsaa_alkane = foyer.Forcefield("xml_files/oplsaa_alkanes.xml")
typed = oplsaa_alkane.apply(compound)
typed.save('test2.top', overwrite=True)


In [ ]:
!cat test2.top

### GMSO enabled Foyer

Foyer has been undergoing development to support GMSO. Interaction with foyer is very similar to the prior example,  where we load in a Foyer forcefield into GMSO. This can be applied to a structure very similarly, using the `gmso.parameterization` suite of tools. This also allows you to load individual forcefields and apply the to seperate substructures of the topology. See the tutorial labeled **tutorials/GMSO-tutorial.ipynb** for more information.

## SMARTS for Non-Atomistic Systems


Thus far, all of our tutorials for mBuild and Foyer have focused on fully atomistic systems, however, MoSDeF is not limited to such systems. 

First, we will demonstrate how to construct a generic coarse-grained polymer system using mBuild, then demonstrate how to define an associated forcefield file.

As done previous in the mBuild tutorials, we will create `Compounds` to describe repeat units in our polymer. Here we will create two `Compounds`, one to describe the central beads (`_A`) and one for the terminal groups (`_B`).

Note, to properly handle non-atomistic types, Foyer expects names to be prefaced by an underscore (e.g., `_A`,`_B`, `_CH2`, etc. ).

In [ ]:
polymer.add_monomer?

In [ ]:
class CentralBead(mb.Compound):
    def __init__(self):
        super(CentralBead, self).__init__()
        
        bead = mb.Particle(pos=[0.0, 0.0, 0.0], name='_A')
        self.add(bead)
        up_port = mb.Port(anchor=bead, orientation=[0, 1, 0], separation=0.05)
        down_port = mb.Port(anchor=bead, orientation=[0, -1, 0], separation=0.05)
        self.add(up_port, label='up')
        self.add(down_port, label='down')
        
class TerminalBead(mb.Compound):
    def __init__(self):
        super(TerminalBead, self).__init__()
        
        bead = mb.Particle(pos=[0.0, 0.0, 0.0], name='_B')
        self.add(bead)

        cap_port = mb.Port(anchor=bead, orientation=[0, 1, 0], separation=0.05)
        self.add(cap_port, label='cap')

class CGPolymer(mb.Compound):
    def __init__(self, chain_length):
        super(CGPolymer, self).__init__()
        
        terminal_bead = TerminalBead()
        last_unit = CentralBead()
        mb.force_overlap(move_this=terminal_bead,
                         from_positions=terminal_bead['cap'],
                         to_positions=last_unit['up'])
        self.add(last_unit, label='_A[$]')
        self.add(terminal_bead, label='up-cap')   
        for _ in range(chain_length - 3):
            current_unit = CentralBead()
            mb.force_overlap(move_this=current_unit,
                             from_positions=current_unit['up'],
                             to_positions=last_unit['down'])
            self.add(current_unit, label='_A[$]')
            last_unit=current_unit
        terminal_bead = TerminalBead()
        mb.force_overlap(move_this=terminal_bead,
                         from_positions=terminal_bead['cap'],
                         to_positions=last_unit['down'])
        self.add(terminal_bead, label='down-cap')
        if chain_length < 3:
            print("Note, the shortest chain this function will make is 3")
cg_polymer = CGPolymer(chain_length=6)
cg_polymer.visualize(color_scheme={'_A': 'blue', '_B': 'red'})


With a simple mBuild code to create CG polymers, let us examine how to define a forcefield file.

In this forcefield, we will assume all particles have the same bonds, angles, and Lennard-Jones sigma, regardless of type, however, we will modify Lennard-Jones epsilon based on chemical context. 

Here we will define the forcefield such that:
- when bead `_A` has 2 bonded neighbors of type `_A`, epsilon = 1.0 
 - atom-type: `cg_A_AA`
 - SMARTS definition: `[_A;X2](_A)(_A)`
- when bead `_A` has 1 bonded neighbor of type `_A` and one of type `_B`, epsilon = 1.25 
 - atom-type `cg_A_AB`
 - SMARTS definition: `[_A;X2](_A)(_B)`
- when bead `_A` has 2 bonded neighbors of type `_B`  epsilon = 1.5
 - atom-type `cg_A_BB`
 - SMARTS definition: `[_A;X2](_B)(_B)`
- when bead `_B` has any neighbor, epsilon = 2.0 regardless of who it is bonded 
 - `atom-type cg_B`
 - SMARTS definition: `[_B;X1]`

The xml forcefield file is shown below:

In [ ]:
cat xml_files/CG_polymer.xml


Let us now atomtype the system.  We will save this to the hoomd xml file format, as it provides a very clear format for viewing the parameterized system.  Note, this format is now deprecated, in place of gsd, which MoSDeF also supports.


Note, because this system does not include dihedral parameters, we must set the argument `assert_dihedral_params` to `False`.  If we did not set this, the code would produce an error stating there are undefined dihedrals.  We now only get warnings instead of an error.   Similar flags can be set for bonds and angles.

In [ ]:
cg_polymer.save("cg.hoomdxml", forcefield_files="xml_files/CG_polymer.xml", overwrite=True, 
                foyer_kwargs={"assert_dihedral_params":False})

cg_polymer.save("cg.gsd", forcefield_files="xml_files/CG_polymer.xml", overwrite=True, 
                foyer_kwargs={"assert_dihedral_params":False})

In [ ]:
!cat cg.hoomdxml

### Saving a reference file. 
To keep track of which parameters were used, we can optionally output a reference file, by passing the `references_file` flag. 

This will produce a BibTeX formatted file containing the references for the parameters used for parameterization of this `Compound` (assuming DOIs have been provided). This fetches the citation information associated with the DOI and in the `note` section lists the atom type names that came from this source.

Here, we explore this functionality by loading a simple methane molecule into an mBuild `Compound` and save using the forcefield we've just defined.

In [ ]:
import mbuild as mb

ch4 = mb.load("C", smiles=True)
ch4.save("CH4.top",
         forcefield_files='xml_files/oplsaa_alkanes.xml',
         foyer_kwargs={"references_file":"ch4.bib"},
         overwrite=True)


In [ ]:
! cat ch4.bib 

## Quick-start

If you are interested in defining your existing forcefield(s) in the Foyer XML format, we currently have a forcefield template hosted on Github to help you get started - https://github.com/mosdef-hub/forcefield_template, that includes some basic scripts for validation and error checking. We also refer interested users to the [OpenMM documentation](http://docs.openmm.org/7.0.0/userguide/application.html#creating-force-fields) for detailed instructions on forcefield creation.